# First Assignment for Data Science Course by Digital House

La inmobiliaria Properati publica periódicamente información sobre ofertas de propiedades para venta y alquiler. 

<b> En este proyecto nos proponemos desarrollar un modelo de regresión que permita predecir el precio en dólares por metro cuadrado de una propiedad. El objetivo final, es desarrollar un tasador automático para ser aplicado a las próximas propiedades que sean comercializadas por la empresa.</b>

Para ello, la empresa le provee de un dataset correspondiente al primer semestre de 2017. 

Será importante tener en cuenta el problema de la <i>influencia espacial en los precios por metro cuadrado</i>. En efecto, es probable que existan diferencias importantes de en las diferentes geografías, barrios y zonas analizadas.


Objetivos de la primera etapa:

● Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.

● Realizar un análisis descriptivo de las principales variables.

● Crear nuevas columnas a partir de las características dadas que puedan tener valor predictivo.

Para la presentación en clase se deben armar algunos slides no técnicos para exponer en no más de 10 minutos (PPT o Google Slides). 
La presentación debe constar:
* una introducción (planteo del problema, la pregunta, la descripción del dataset, etc.) 
* un desarrollo de los análisis realizados (análisis descriptivo, análisis de correlaciones preliminares, visualizaciones preliminares) 
* una exposición de los principales resultados y conclusiones.

## Introducción

Problema: automatizar el proceso de predecir el precio por metro cuadrado de una propiedad en USD, creando así un tasador automático.

Entonces, ¿a qué features nos debemos basar para predecir el valor?


<img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DS0103EN/labs/images/lab1_fig2_datascience_methodology_flowchart.png" width = 500> 

Vamos a definir el enfoque analítico para resolver el problema, identificando qué tipo de patrones serán necesarios para abordar la cuestión de manera más efectiva. Aquí utilizaremos técnicas estadísticas y de aprendizaje automático.

## Preparación de datos

In [18]:
import pandas as pd
import numpy as np
df=pd.read_csv("properatti.csv")

In [19]:
df.shape

(121220, 26)

In [20]:
df.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [21]:
df.rename(columns={"Unnamed: 0":"a","operation":"operacion","property_type":"tipo_inmueble","place_name":"barrio","place_with_parent_names":"localizacion","country_name":"argentina","state_name":"localidad","geonames_id":"id_geoname","lat-lon":"coordenadas","lat":"latitud","lon":"longitud","price":"precio","currency":"moneda","price_aprox_local_currency":"precio_pesos","price_aprox_usd":"precio_usd","surface_total_in_m2":"sup_total","surface_covered_in_m2":"sup_cubierta","price_usd_per_m2":"precio_usd_sup","price_per_m2":"precio_pesos_sup","floor":"nivel","rooms":"ambientes","expenses":"expensas","properati_url":"url","description":"observaciones","title":"titulo","image_thumbnail":"url_imagen"}, inplace=True)

In [22]:
df.drop(columns="a", inplace=True)

In [23]:
df.head()

,operacion,tipo_inmueble,barrio,localizacion,argentina,localidad,id_geoname,coordenadas,latitud,longitud,...,sup_cubierta,precio_usd_sup,precio_pesos_sup,nivel,ambientes,expensas,url,observaciones,titulo,url_imagen
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [24]:
df.describe()

,id_geoname,latitud,longitud,precio,precio_pesos,precio_usd,sup_total,sup_cubierta,precio_usd_sup,precio_pesos_sup,nivel,ambientes,expensas
count,1.025030e+05,69670.000000,69670.000000,1.008100e+05,1.008100e+05,1.008100e+05,81892.000000,101313.000000,68617.000000,8.765800e+04,7899.000000,47390.000000,1.426200e+04
mean,3.574442e+06,-34.626210,-59.266290,4.685259e+05,4.229397e+06,2.397006e+05,233.795328,133.050181,2160.086916,6.912216e+03,17.452336,3.080840,5.009234e+03
std,3.541306e+05,1.980936,2.299922,2.260101e+06,6.904714e+06,3.913239e+05,1782.222147,724.351479,2759.288621,2.837864e+04,120.243621,1.860773,1.204403e+05
min,3.427208e+06,-54.823985,-75.678931,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.600000,1.510204e+00,1.000000,1.000000,1.000000e+00
25%,3.430234e+06,-34.669065,-58.727040,1.100000e+05,1.583309e+06,8.973388e+04,50.000000,45.000000,1218.181818,1.550000e+03,1.000000,2.000000,1.000000e+03
50%,3.433910e+06,-34.597985,-58.480128,1.850000e+05,2.558452e+06,1.450000e+05,84.000000,75.000000,1800.000000,2.213115e+03,3.000000,3.000000,2.000000e+03
75%,3.836668e+06,-34.441299,-58.395908,4.200000e+05,4.675792e+06,2.650000e+05,200.000000,150.000000,2486.411765,3.355549e+03,6.000000,4.000000,4.000000e+03
max,6.948895e+06,4.545843,-53.733330,6.500000e+08,8.212711e+08,4.654544e+07,200000.000000,187000.000000,206333.333333,4.000000e+06,3150.000000,32.000000,1.000150e+07


In [25]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import datasets

In [34]:
nulos=df.isnull().sum()
pd.DataFrame(nulos)

,0
operacion,0
tipo_inmueble,0
barrio,23
localizacion,0
argentina,0
localidad,0
id_geoname,18717
coordenadas,51550
latitud,51550
longitud,51550


<h5> Completando valores nulos 

De dónde sacar los datos:

    -barrio: localidad
    -id_geoname: ¿qué utilidad tiene?
    -coordenadas, latitud y longitud: ver
    -precios usd es lo más completo, nos sirve para completar los 10000 faltantes dentro de superficie
    -de que sirve la imagen????? al día de hoy no podemos codificar imagenes y viceversa ----> eliminar
    -
    -
    -
    -

In [47]:
nulos.loc[["barrio","id_geoname","coordenadas","latitud","longitud","precio","moneda","precio_pesos","precio_usd","precio_pesos_sup","precio_usd_sup","sup_total","sup_cubierta","nivel","ambientes","expensas","url_imagen"]]

barrio                  23
id_geoname           18717
coordenadas          51550
latitud              51550
longitud             51550
precio               20410
moneda               20411
precio_pesos         20410
precio_usd           20410
precio_pesos_sup     33562
precio_usd_sup       52603
sup_total            39328
sup_cubierta         19907
nivel               113321
ambientes            73830
expensas            106958
url_imagen            3112
dtype: int64

In [44]:
df.observaciones[0]

'2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO, REFACCIONADO A NUEVO, PATIO GRANDE, CON LAVADERO, LIVING COMEDOR CON COCINA INTEGRADA, ARTEFACTO DE COCINA, ALACENA, BAÑO COMPLETO, Y DORMITORIO. TODO EN EXCELENTE ESTADO, PARA HABITAR.NO ES APTO CREDITO Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).'

In [45]:
df.observaciones[10]

'Hermosa casa sin terminar de 3 dormitorios, baño, living comedor, cocina, cochera.Techo de Madera. Con escritura y servicios. Lote de 1514 metros.                     Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).'

In [46]:
df.observaciones[1000]

'Sobre terreno  de 4.700 m2 con vista a campo abierto y monte, casa de 110m2 , con 3 dormitorios y 2 baños,cocina integrada, living comedor con chimenea. Casa de depósito.  Pileta con filtro. Gran arboleda, terreno parquizado .                  Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).'

<h3>Boludeses mías


http://localhost:8888/notebooks/Documents/data%20science%20DH/IBM%20cuadernos%20J%20coursera/data%20methodology/DS0103EN-4-4-1-From-Modeling-to-Evaluation-v2.0.ipynb

http://localhost:8888/notebooks/Documents/data%20science%20DH/IBM%20cuadernos%20J%20coursera/data%20visualization/DV0101EN-1-1-1-Introduction-to-Matplotlib-and-Line-Plots-py-v2.0.ipynb
        
        
        ingredients = list(recipes.columns.values)

print([match.group(0) for ingredient in ingredients for match in [(re.compile(".*(rice).*")).search(ingredient)] if match])

In [ ]:
recipes["country"].value_counts() # frequency table

In [ ]:
recipes["cuisine"] = recipes["cuisine"].str.lower()

In [ ]:
recipes.loc[recipes["cuisine"] == "irish", "cuisine"] = "uk-and-irish"

In [ ]:
cuisines = recipes.groupby("cuisine").mean()

In [ ]:
def print_top_ingredients(row):
    print(row.name.upper())
    row_sorted = row.sort_values(ascending=False)*100
    top_ingredients = list(row_sorted.index.values)[0:num_ingredients]
    row_sorted = list(row_sorted)[0:num_ingredients]

    for ind, ingredient in enumerate(top_ingredients):
        print("%s (%d%%)" % (ingredient, row_sorted[ind]), end=' ')
    print("\n")

# apply function to cuisines dataframe
create_cuisines_profiles = cuisines.apply(print_top_ingredients, axis=1)

In [ ]:
random.seed(1234) # set random seed
bamboo_test = bamboo.groupby("cuisine", group_keys=False).apply(lambda x: x.sample(sample_n))

bamboo_test_ingredients = bamboo_test.iloc[:,1:] # ingredients
bamboo_test_cuisines = bamboo_test["cuisine"] # correspond